# Build a Word2Vec Model from wikipedia dump

including dictionary of all words and domain specific training
https://dumps.wikimedia.org/dewiki/latest/

In [1]:
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.corpora import MmCorpus
import gensim
import multiprocessing
from gensim.models.word2vec import Word2Vec


C:\Users\cb2lehk\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [28]:
%time wiki = WikiCorpus('dewiki-latest-pages-articles.xml.bz2')

Wall time: 1h 11min 6s


In [ ]:
wiki.save('wiki.corpus')

sample_text = wiki.sample_texts(5, seed=1, length=10)

print(sample_text)

In [31]:
%time MmCorpus.serialize('wiki_en_vocab200k.mm', wiki)

Wall time: 1h 7min 48s


In [32]:
mm = MmCorpus('wiki_en_vocab200k.mm')

### Convert to text-file for further preprocessing

sentences = list(wiki.get_texts())
with open("wiki_text.txt", "w") as output:
    output.write(str(sentences))

In [33]:
class MySentences(object): 
    def __iter__(self): 
        for text in wiki.get_texts(): 
            #yield [word.decode() for word in text] 
            yield [word for word in text] 
sentences = MySentences() 

In [34]:
sentences = MySentences()
params = {'size': 50, 'window': 10, 'min_count': 40, 
          'workers': max(1, multiprocessing.cpu_count() - 1), 'sample': 1e-3,}
word2vec = Word2Vec(sentences, **params)
word2vec.save('wiki.word2vec_50.model')

In [25]:
wiki_woerterbuch = {}
for word, vocab_obj in new_model.wv.vocab.items():
    wiki_woerterbuch[word] = vocab_obj.count

In [26]:
import pickle
pickle.dump(wiki_woerterbuch, open('wiki_woerterbuch.p', 'wb'))

In [ ]:
word2vec.wv.most_similar('bank')

finish initial train

In [ ]:
word2vec.most_similar(positive=['frau', 'könig'], negative=['mann'], topn=1)

## Train the model further on domain specific content 

In [2]:
import pickle
domain = pickle.load(open('text_domain.p', 'rb'))

In [7]:
new_model = gensim.models.Word2Vec.load('wiki.word2vec_50.model')

In [9]:
new_model.build_vocab(domain, update=True)

In [10]:
new_model.train(domain,len(domain),epochs=10)

103711503

In [11]:
new_model.most_similar('karte')

[('girokarte', 0.8161906003952026),
 ('kreditkarte', 0.7242053747177124),
 ('girocard', 0.7163479328155518),
 ('Karte', 0.7050700187683105),
 ('Überweisung', 0.667661726474762),
 ('Mietkaution', 0.650923490524292),
 ('MaestroCard', 0.6496913433074951),
 ('sonderkarte', 0.6426790952682495),
 ('passagierliste', 0.6392239928245544),
 ('schiffsdaten', 0.6371652483940125)]

In [4]:
new_model.save('wiki.word2vec_50.model')